<a href="https://colab.research.google.com/github/chandralabs/tamil-llama/blob/main/Copy_of_Dravidian_LLaMA_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tamil/Telugu/Malayalam LLaMA Demo

This is an interactive colab notebook where you can easily interact with the Indic LLaMA models developed by [@abhinand](https://www.linkedin.com/in/abhinand-05/).

To dive deep into the development and capabilities of this model, please read the [research paper](https://arxiv.org/abs/2311.05845) and the [introductory blog post](https://abhinand05.medium.com/breaking-language-barriers-introducing-tamil-llama-v0-2-and-its-expansion-to-telugu-and-malayalam-deb5d23e9264) that outlines our journey and the model's potential impact.

> **Note:** This model is based on the Tamil LLaMA series of models. The GitHub repository remains the same - [https://github.com/abhinand5/tamil-llama](https://github.com/abhinand5/tamil-llama). The base models and the updated code for Tamil LLaMA v0.2 (which this work is based on) will be released soon.

> **Important:** Make sure you are connected to a GPU runtime, this doesn't work on CPU.

## Available Models

- [abhinand/tamil-llama-7b-instruct-v0.2](https://huggingface.co/abhinand/tamil-llama-7b-instruct-v0.2)
- [abhinand/telugu-llama-7b-instruct-v0.1](https://huggingface.co/abhinand/telugu-llama-7b-instruct-v0.1)
- [abhinand/malayalam-llama-7b-instruct-v0.1](https://huggingface.co/abhinand/malayalam-llama-7b-instruct-v0.1)

## Quick Start Guide

1. **Select Your Language Model**: Begin by choosing your desired Language Model from the `LLM_LANGUAGE` dropdown in the Initial Setup section. Typically, there's no need to adjust other settings here. Simply execute the cell to proceed.

    Please note that downloading and loading the model onto the GPU memory may take some time.

2. **Interact with the Model**: After the successful execution of cell 1, you're ready to interact with the model. Input your system prompt and queries, then run the cell to receive the model's responses.

    Remember, this notebook is designed for single-turn dialogues. To continue the interaction, modify your inputs and system prompts, and run the cell again to view the model's new outputs.

> **⚠️ Important Note:** Please set the system prompt in the language you are conversing in or do not set it at all, for example if you're coversing in Tamil with Tamil LLaMA, the system prompt has to be in Tamil not English (or empty). This is because the models weren't trained to handle cross-lingual instructions. Language mismatch in system prompt only confuses the model and leads to hallucination.


If Colab prompts you to give access to your `HF_TOKEN` secret variable, feel free to **cancel it**, it is not required for this demo because we're dealing with public models.

![image.png](https://i.postimg.cc/3R88BpLX/tmp.png)



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# @title # Initial Setup

# !pip install torch==2.0.1 -q
!pip install transformers accelerate bitsandbytes trl peft datasets huggingface_hub sentencepiece -qU

# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

import os
# from google.colab import userdata
# HF_TOKEN_VAR = "HF_TOKEN" # @param {type:"string"}
# hf_token = userdata.get('HF_TOKEN')

import torch
from transformers import LlamaForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from huggingface_hub import snapshot_download

import warnings
warnings.filterwarnings('ignore')

def is_directory_empty_or_nonexistent(directory_path):
    if not os.path.exists(directory_path):
        return True

    if os.path.isdir(directory_path) and not os.listdir(directory_path):
        return True

    return False


def get_model_name(language):
    if language == "tamil":
        return "abhinand/tamil-llama-7b-instruct-v0.2"
    elif language == "telugu":
        return "abhinand/telugu-llama-7b-instruct-v0.1"
    elif language == "malayalam":
        return "abhinand/malayalam-llama-7b-instruct-v0.1"
    else:
        return None

LLM_LANGAUGE = "tamil" # @param ["tamil", "telugu", "malayalam"] {type:"string"}
MODEL_NAME = get_model_name(LLM_LANGAUGE)
MODEL_DIR = "llama"
REVISION = "main" # @param {type:"string"}

if is_directory_empty_or_nonexistent(MODEL_DIR):
    snapshot_download(
        repo_id=MODEL_NAME, local_dir=MODEL_DIR,
        local_dir_use_symlinks=False, revision=REVISION#, token=hf_token
    )

LOAD_IN_8_BIT = True # @param {type:"boolean"}
USE_BFLOAT16 = False # @param {type:"boolean"}

model = LlamaForCausalLM.from_pretrained(
    MODEL_DIR,
    load_in_8bit=LOAD_IN_8_BIT,
    torch_dtype=torch.bfloat16 if USE_BFLOAT16 else torch.float16,
    device_map={"": 0},
    # local_files_only=False
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

generation_config = GenerationConfig(
    temperature=0.6,
    # top_k=50,
    top_p=0.9,
    repetition_penalty=1.1,
    num_return_sequences=1,
    # num_beams=1,
    max_length=512,
    # eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    max_new_tokens=256,
)

inf_pipeline = pipeline("conversational", model=model, tokenizer=tokenizer)


def format_instruction(system_prompt, question, return_dict=False):
	if system_prompt is None:
		messages = [
			{'content': question, 'role': 'user'},
		]
	else:
		messages = [
			{'content': system_prompt, 'role': 'system'},
			{'content': question, 'role': 'user'},
		]

	if return_dict:
		return messages

	prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

	return prompt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.55k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, Trainer

# Load the existing model and tokenizer
model = LlamaForCausalLM.from_pretrained("abhinand/tamil-llama-7b-instruct-v0.2")
tokenizer = LlamaTokenizer.from_pretrained("abhinand/tamil-llama-7b-instruct-v0.2")

# Load your new dataset
new_dataset = "/content/drive/MyDrive/tamil-llama-education/Mastersheet_TamilLLMAProject.xlsx" # Load your new dataset here

# Combine the existing and new datasets
combined_dataset = existing_dataset + new_dataset

# Train the model
trainer = Trainer(
    model=model,
    train_dataset=combined_dataset,
    eval_dataset=validation_dataset,
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    warmup_steps=100
)
trainer.train()

# Save the updated model
model.save_pretrained("abhinand/tamil-llama-7b-instruct-v0.2.1")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
# @title # Chat with Model

# @markdown > **⚠️ Important Note:** Please set the system prompt in the language you are conversing in or do not set it at all, for example if you're coversing in Tamil with Tamil LLaMA, the system prompt has to be in Tamil not English (or empty). This is because the models weren't trained to handle cross-lingual instructions. Language mismatch in system prompt only confuses the model and leads to hallucination.

# @markdown ## Generation Config

temperature = 0.9 # @param {type:"slider", min:0, max:1, step:0.1}
repetition_penalty = 1.3 # @param {type:"slider", min:1, max:1.5, step:0.05}
max_new_tokens = 384 # @param {type:"slider", min:128, max:1024, step:64}

# @markdown ---
# @markdown ## Enter your input here

SYSTEM_PROMPT = "You are an AI assistant who follows instructions extremely well. Do your best your best to help." # @param {type:"string"}
INPUT = "Tell me all the vowels in Tamil?" # @param {type:"string"}

instruction = format_instruction(
    system_prompt=SYSTEM_PROMPT,
    question=INPUT,
    return_dict=True,
)

output = inf_pipeline(
    instruction,
    temperature=temperature,
    max_new_tokens=max_new_tokens,
    repetition_penalty=repetition_penalty
)
output

Conversation id: fe6106c3-cb94-41da-a2da-fec7722a048d
system: You are an AI assistant who follows instructions extremely well. Do your best your best to help.
user: Tell me all the vowels in Tamil?
assistant: In Tami, there would be different sets of consonants for some words and this may include letters like h (in English) that represent a certain sound or syllable in those specific contexts too but not necessarily considered as part of common letter groupings with other similar sounds. The unvowed alphabets used only within these special combinations can typically have various meanings depending on their positioning and formation in tamil language text.